# Database Creator

In [ ]:
# All imports go here

import os
import sys
import requests
from bs4 import BeautifulSoup as bsoup
import csv
import wget

## Combine interest keywords into a single keyword dictionary

In [ ]:
def add_to_results(data, results):
    interests = data[-1]
    for interest in interests:
        # sanitise interest by changing space to
        # _ and converting all to lower case
        interest_sanitised = interest.replace('/', '_').replace(' ', '_').lower()

        # create key if does not exist
        if interest_sanitised not in results:
            results[interest_sanitised] = [data]
        else:
            results[interest_sanitised].append(data)
    return results

## IISER-K Retriever

In [ ]:
def iiserk():
    url = "https://www.iiserkol.ac.in/web/en/people/faculty/dps/#gsc.tab=0"
    soup = bsoup(requests.get(url).content, features='lxml')
    results = {}
    for tag in soup.findAll('b', text="Department:"):
        parent = tag.findPrevious('td')
        # print (parent.findChild('a').findNext())
        name = parent.findChild('a', attrs={'style':"color:#0976e4;"}).text.strip()
        desig = parent.findChild('b', text="Department:").findPrevious().previousSibling.strip()
        try:
            interests_string = parent.findChild('b', text="Research Area:").nextSibling
            delim = ';' if ';' in interests_string else ','
            interests = [s.strip() for s in interests_string.split(delim)]
        except:
            interests = []
        homepage = "https://www.iiserkol.ac.in" + tag.findPrevious('a')['href']
        data = [name, "IISER-K", desig, homepage, interests]
        results = add_to_results(data, results)
    return results

## Google Scholar Retriever

In [ ]:
def google_scholar():
    
    # URL to search for scientists. The keyword is 'physic', which will hopefully 
    # return most of the physics scientists. The Indian search is enforced through
    # the domains .ac.in and .res.in. Currently searches only through first page,
    # needs to be extended to all pages
    
    url = "https://scholar.google.co.in/citations?hl=en&view_op=search_authors&mauthors=physic+%2B+.ac.in+%7C+.res.in&btnG="
       
    page = requests.get(url)
    soup = bsoup(page.content, features='lxml')
    results = {}
    
    count = 1
    while soup != []:
        print ("G Scholar: Page", count)
        count += 1
        
        tags = soup.findAll('h3', attrs={'class': "gs_ai_name"})
        if tags == []:
            print ("There's a problem.")
        for tag in tags:

            # obtain name, affiliation, interests and homepage (if exists)
            name = tag.text
            link = "https://scholar.google.com"+tag.next['href']
            author_soup = bsoup(requests.get(link).content, features='lxml')
            affil_tag = author_soup.find('div', attrs={'class':"gsc_prf_il"})
            affil = affil_tag.text
            try:
                homepage = author_soup.find('a', text = "Homepage")['href']
            except:
                homepage = ""
            interests = [child.text for child in author_soup.find('div', attrs={'class':"gsc_prf_il", 'id':"gsc_prf_int"}).findChildren()]

            data = [name, affil, affil, homepage, ', '.join(interests)]

            # append data for this scientist to the dictionary
            results = add_to_results(data, results)
            
        try:
            next_btn = soup.find('button', attrs={'aria-label': "Next", 'class': "gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx"})
            url = 'https://scholar.google.co.in' + str(next_btn['onclick']).replace('window.location=\'', '')[:-1]
            index = url.find("\\x")
            while index >= 0:
                hexa = url[index+2:index+4]
                actual = bytearray.fromhex(hexa).decode()
                url = url.replace("\\x"+hexa, actual)
                index = url.find("\\x")
            page = requests.get(url)
            soup = bsoup(page.content, features='lxml')
        except:
            soup = []
        
    return results

## IISc Retriever

In [ ]:
def iisc():
    results = {}
    url_aap = "http://www.physics.iisc.ac.in/~jap/people-fac.html"
    fname = "file.html"
    os.system("wget " + url_aap + " -O " + fname + " >/dev/null 2>&1")
    soup = bsoup(open(fname, 'r'))
    for tag in soup.findAll('div', attrs={'class': "about-veno"}):
        name = tag.findNext('h3').text
        desig = tag.findNext('h4').text
        interests = str(list(tag.findNext('h5', text='Research Interests:').next_siblings)[0]).replace('\n', '').strip().split('.')
        homepage = tag.findNext('span', text='Web:').findNext('a')['href']
        data = [name, "IISc", desig, homepage, ', '.join(interests)]

        results = add_to_results(data, results)
    os.system("rm " + fname)
    return results

# Dump results into files

**WARNING: Existing files are overwritten at present.**

In [102]:
# headings of each CSV file
headings = ['Name', 'Institute', 'Designation', 'Homepage', 'Interests']

# path of database folder, inside which all the
# csv files will live. Folder is automatically
# created if it does not exist.
folder = "./physics_database/"
if not os.path.exists(folder):
    os.mkdir(folder)

# obtain dictionaries from all existing retrievers
results_arr = []
results_arr.append(iiserk())

# write results of each dictionary into each csv file
for results in results_arr:
    # print (results)
    for key in results:
        # file is opened in append mode 'a', this ensures
        # that existing files are not overwritten
        csv_w = csv.writer(open(folder+key+".csv", 'w'), delimiter='\t')
        data = results[key]
        csv_w.writerow(headings)
        csv_w.writerows(data)
        
        for cat, val in categories.items():
            if key in val:
                csv_ww = csv.writer(open(folder_cat+cat+".csv", 'w'), delimiter='\t')
                csv_ww.writerow(headings)
                csv_ww.writerows(data)

In [98]:
categories = {
    'cond_mat': ["condensed_matter_physics_(experimental)",
                 "carrier_dynamics_in_semiconductors",
                 "quantum_magnetism",
                 "strongly_correlated_electron_systems_and_magnetooptics",
                "experimental_condensed_matter_physics",
                "bose-einstein_condensates",
                "cold_fermions",
                "quantum_condensed_matter_physics",
                "strongly_correlated_electron_systems",
                "condensed_matter_(experiment)",
                "emergent_phenomena_in_low-dimensional_quantum_systems",
                "condensed_matter_theory",
                "soft_condensed_matter",
                "soft_condensed_matter_physics",
           ],
    'optics': ["spectroscopy", 
          "ultrafast_optical_spectroscopy", 
          "nonlinear_optical_phenomena", 
          "optics_and_photonics", 
          "photoelectron-photoion_imaging_spectroscopy", 
          "optical_physics", 
          "ultrafast_spectroscopy", 
          "thz_spectroscopy", 
          "thz_metamaterials", 
          "nonlinear_optics_thz",
          "plasmonics_and_nano_optics",
          "polarization_optics",
          "biophotonics"
         ],
    'qiqc': ["quantum_information_processing",
       "quantum_information_and_quantum_computation",
       "quantum_computation_and_quantum_information",
       ],
    'high_energy': ["high_energy_physics",
              ],
    'grav_cosmol': ["gravitation_&amp", 
           "cosmology", 
           "gravitation_and_cosmology_(classical_and_quantum)", 
           "cosmology_and_its_connections_to_particle_physics",
          ],
    'quant_mech': ["quantum_phenomena",
              "non-equilibrium_quantum_dynamics",
              "quantum_transport",
             ],
    'nld': ["non-linear_dynamics",
       "nonlinear_optical_phenomena",
       "nonlinear_dynamics",
      ],
    'biophys': ["mathematical_and_theoretical_biology", 
           "computational_biology", 
           "biophysics_and_complex_systems",
           "theoretical_biological_physics",
           "biophotonics",
          ],
    'astro': ["space_science",
         "astrophysics",
         "astrophysical_magnetohydrodynamics",
        ],
    'material_sc': ["polymer_physics", 
                    "quantum_theory_of_functional_materials",
                   ],
    'stat_phys': ["statistical_physics",
                  "nonequilibrium_statistical_physics",
            ],
    'others': ["open_quantum_systems",
               "nmr",
               "physics",
               "polyelectrolyte_physics",
               "low_energy_electron-molecule_collisions",
               "surface_science",
               "weak_measurements",
               "magnetism_in_mesoscopic_systems_and_spintronics_application",
               "field_theory_and_wavelet_transform",
             ]
}


# categories = [cond_mat, optics, qiqc, high_energy, grav_cosmol, quant_mech, nld, biophys, astro, material_sc, stat_phys, others]
category_file = "./category_file"
folder_cat = "./physics_database/categories/"
if not os.path.exists(folder_cat):
    os.mkdir(folder_cat)
with open(category_file, 'w') as f:
    for cat, val in categories.items():
        if not os.path.exists(folder_cat+cat+".csv"):
            os.mknod(folder_cat+cat+".csv")
        f.write(cat + ':' + ';'.join(val) + '\n')
f.close()